# Project 2
## Joshua Anderson

## Introduction to Fantasy Football
Fantasy football is a game where a group of participants serve as general managers of their own virtual football team. Managers build their roster (team) through a draft process in which they select real NFL players. Each team is completely unique because a player can only be on one manager's roster. The performance of a team is determined by the number of points generated by its players, and a player's points are a direct function of their real world performance statistics. As a manager, your goal is to draft the team that generates the most points.

Although there are many different fantasy draft mechanisms, the mechanism we are interested in is an auction draft. An auction draft can be thought of as a set of sequential player auctions where the highest bidding manager wins the player and pays their bid. Each manager is endowed with a budget of 200 to spend in the draft. You have been provided a data set that includes an expected price for each player. You can think of this as the price you must pay to win the player. During the draft you will select a total of 15 players.

In addition to drafting the team, a manager is also responsible for choosing a weekly starting lineup. A starting lineup consists of 1 QB, 3 WR, 2 RB, 1 TE, 1 K, and 1 DEF. Any player not selected to be in your starting lineup is on your bench. Your team generates points from the players you choose to be in your starting lineup. In order to simplify the problem, the optimization will not have to choose a starting lineup for each week. Instead, we will assume that you will start the players with the highest season long points every week and discount a portion of the points generate by your bench players. This means you will receive all points generate by players chosen as starters but only a portion of the points generate by players on your bench. The discount factor to use for bench player is defined in 9 below.

You can download the dataset as a csv file from here:

[Project 2 Data](https://drive.google.com/file/d/1AKgAsyHXOZLRh0urpd4Ng_K_c3g3QA6m/view?usp=sharing) (Links to an external site.)

 

## General Parameters
- You have a maximum budget of 200.
- A roster consists of exactly 15 players.
- A roster has 1 QB slot, 3 WR slots, 2 RB slots, 1 TE slot, 1 K slot, 1 DEF slot, and 6 Bench slots.
- Any QB, WR, RB, TE, K, DEF slot can only contain a player of that type.
- A Bench slot can contain any player type.
- A player on your roster can only be assigned to one slot.
- You want at most 2 QB and 2 TE on your roster.
- You want at most 1 K and 1 DEF on your roster.
- A player on your bench will only generate a fraction of their projected points. A bench player should generate ½d of their projected points where d is their depth at their position. For example if you have RB1, RB2, RB3, and RB4 on your team with projected points P1, P2, P3, and P4 the expected points from these players would be $P1 + P2 + ½ * P2 + ¼ * P3$

## Questions to answer
- Who would you choose to maximize points while staying within budget?
- How many points is your team expected to generate?
- How much of your budget did you spend?

In [60]:
import pandas as pd
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')
data = pd.read_csv("fantasy_football_data.csv")

def objective(i):
    name = data.Name[i]
    pos = data.Position[i]
    values = []
    for depth in range(depth_lims[pos]):
        # if player is a starter, use the normal points
        if starter_lims[pos] <= depth+1:
            values.append(deltas[pos][(name, depth)] * points[i])
        # if player is a bench player, use the discounted points
        else:
            values.append(deltas[pos][(name, depth)] * points[i] * (1/pow(2, depth+1-starter_lims[pos])))
    return sum(values)

budget = 200
roster_len = 15

starter_lims = {
    'QB': 1,
    'WR': 3,
    'RB': 2,
    'TE': 1,
    'K': 1,
    'DEF': 1
}

depth_lims = {
    'QB': 2,
    'WR': 7,
    'RB': 7,
    'TE': 2,
    'K': 1,
    'DEF': 1
}

prices = list(data.Price)
points = list(data.ProjectedPoints)
positions = list(data.Position)

# create dictionary of every position containing a sub dictionary of every player at every possible depth at their position
deltas = {}
for pos in set(positions):
    deltas[pos] = ({(data.Name[i],depth): solver.IntVar(0, 1, data.iloc[i, 0] + "_" + str(depth)) for i in data.index[data.Position == pos] for depth in range(depth_lims[pos])})

# use the objective function declared above
solver.Maximize(sum([objective(i) for i in data.index]))

# Constraints

# roster of 15
solver.Add(sum([deltas[pos][(data.Name[i], depth)] for pos in set(positions) for i in data.index[data.Position == pos] for depth in range(depth_lims[pos]) ]) == roster_len)

# stay under budget
solver.Add(sum([deltas[pos][(player, depth)] * data.Price[data.Name == player].iloc[0] for pos in set(positions) for player in set(data.Name[data.Position == pos]) for depth in range(depth_lims[pos])]) <= budget)

for pos in set(positions):
    solver.Add(sum([deltas[pos][(player, 0)] for player in set(data.Name[data.Position == pos])]) >= 1) # must have a starter in each position
    solver.Add(sum([deltas[pos][(player, 0)] for player in set(data.Name[data.Position == pos])]) <= starter_lims[pos]) # limit number of starters in each position
    solver.Add(sum([deltas[pos][k] for k in deltas[pos].keys()]) <= depth_lims[pos]) # max per position

    # only 1 player at each depth for each position
    for depth in range(depth_lims[pos]):
        solver.Add(sum([deltas[pos][(player, depth)] for player in set(data.Name[data.Position == pos])]) <= 1)

for player in set(data.Name):
    pos = data.Position[data.Name == player].iloc[0]
    solver.Add(sum([deltas[pos][(player, depth)] for depth in range(depth_lims[pos])]) <= 1) # cannot have a player more than once



status = solver.Solve()

if status == solver.OPTIMAL:
    starters = []
    bench = []
    for pos in set(positions):
        for k in deltas[pos].keys():
            if deltas[pos][k].solution_value() == 1:
                if k[1] < starter_lims[pos]:
                    starters.append(k)
                else:
                    bench.append(k)

In [71]:
print("This team I would use to maximize points but stay in budget!")
print("Starters:")
for player in starters:
    print(f"{player[0]} - {data.Position[data.Name == player[0]].iloc[0]}")
print()

print("Bench:")
for player in bench:
    print(f"{player[0]} - {data.Position[data.Name == player[0]].iloc[0]}")
print()

print("Here is how many points I would be projected to get!")
starter_points = sum([data.ProjectedPoints[data.Name == player[0]].iloc[0] for player in starters])
bench_points = 0
for player in bench:
    pos = data.Position[data.Name == player[0]].iloc[0]
    bench_points += data.ProjectedPoints[data.Name == player[0]].iloc[0] * (1 / pow(2, player[1]+1 - starter_lims[pos]))
print(starter_points + bench_points)
print()

print("Here is how much of my budget I used!")
budget_used = sum([data.Price[data.Name == player[0]].iloc[0] for player in (starters + bench)])
print(budget_used)

This team I would use to maximize points but stay in budget!

Starters:
Coby Fleener - TE
Adam Vinatieri - K
Le'Veon Bell - RB
Terrance West - RB
Antonio Brown - WR
Michael Crabtree - WR
Rishard Matthews - WR
Ben Roethlisberger - QB
Tampa Bay Buccaneers - DEF

Bench:
Tevin Coleman - RB
C.J. Anderson - RB
Spencer Ware - RB
Jonathan Stewart - RB
Danny Woodhead - RB
Tyrod Taylor - QB

Here is how many points I would be projected to get!
1616.875

Here is how much of my budget I used!
200.0
